In [12]:
from einops import rearrange
import torch
from torch import einsum

In [13]:
einsum?

Signature: einsum(*args: Any) -> torch.Tensor
Docstring:
einsum(equation, *operands) -> Tensor

Sums the product of the elements of the input :attr:`operands` along dimensions specified using a notation
based on the Einstein summation convention.

Einsum allows computing many common multi-dimensional linear algebraic array operations by representing them
in a short-hand format based on the Einstein summation convention, given by :attr:`equation`. The details of
this format are described below, but the general idea is to label every dimension of the input :attr:`operands`
with some subscript and define which subscripts are part of the output. The output is then computed by summing
the product of the elements of the :attr:`operands` along the dimensions whose subscripts are not part of the
output. For example, matrix multiplication can be computed using einsum as `torch.einsum("ij,jk->ik", A, B)`.
Here, j is the summation subscript and i and k the output subscripts (see section below for

In [14]:
rearrange?

Signature:
rearrange(
    tensor: Union[~Tensor, List[~Tensor]],
    pattern: str,
    **axes_lengths: Any,
) -> ~Tensor
Docstring:
einops.rearrange is a reader-friendly smart element reordering for multidimensional tensors.
This operation includes functionality of transpose (axes permutation), reshape (view), squeeze, unsqueeze,
stack, concatenate and other operations.

Examples:

```python
# suppose we have a set of 32 images in "h w c" format (height-width-channel)
>>> images = [np.random.randn(30, 40, 3) for _ in range(32)]

# stack along first (batch) axis, output is a single array
>>> rearrange(images, 'b h w c -> b h w c').shape
(32, 30, 40, 3)

# stacked and reordered axes to "b c h w" format
>>> rearrange(images, 'b h w c -> b c h w').shape
(32, 3, 30, 40)

# concatenate images along height (vertical axis), 960 = 32 * 30
>>> rearrange(images, 'b h w c -> (b h) w c').shape
(960, 40, 3)

# concatenated images along horizontal axis, 1280 = 32 * 40
>>> rearrange(images, 'b h w c -

In [10]:
# torch.randn_like() — creates a tensor of random numbers with the same shape as another tensor
# Create a random matrix of shape (64, 256, 32)
x = torch.randn(64, 256, 32)
x.shape

torch.Size([64, 256, 32])

In [16]:
x_new = rearrange(x, 'n c ( h w ) -> ( n h ) c w', h=8)
x_new.shape

torch.Size([512, 256, 4])

In [ ]:
x_reversed = rearrange(x_new, '( n h ) c w -> n c ( h w )', h=8)
x_reversed.shape

torch.Size([64, 256, 32])

: 